# Prerequisites

1. Start the treasure-hunt game server and frontend using `docker compose up`
2. Observe the status of the game by opening http://localhost:8089/ in a browser

- It gets the current status of the game from the server using websockets, so just leave the browser open and follow through this notebook
- You can play the game in the browser (the buttons are just for debugging though)

## Setup OpenAI

In [1]:
# getenv
from dotenv import load_dotenv
import os
import openai

load_dotenv()  # Load environment variables from .env file
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

# Create model
Use caching to save 💰

In [2]:
import langchain
from langchain.cache import InMemoryCache
from langchain.llms.fake import FakeListLLM
from langchain.chat_models import ChatOpenAI

# model="text-curie-001"
model="gpt-3.5-turbo"
# model=""
llm = ChatOpenAI(model_name=model, temperature=0.7)
#llm = FakeListLLM(responses=["UP", "DOWN"])

langchain.llm_cache = InMemoryCache()

## Chain to describe the game

In [3]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

system_message = """
You are on a 2D-grid and can see on the current and directly to adjacent cells. 
Goal: You need to find and pick up the treasure. Specify only the command for a given situation.
You can only pick up the treasure if the current field has the treasure.
Do NOT start the game again until you have found the treasure.

Descriptions: w=wall, g=ground, t=treausre
Available commands: UP DOWN LEFT RIGHT PICKUP_TREASURE
Input command:
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
human_template="Situation: current:{current}, up:{up}, down:{down}, left:{left}, right:{right}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


In [4]:
situation = {'current':'g','up':'t','down':'g','left':'g','right':'w'}
llm(chat_prompt.format_prompt(**situation).to_messages())

AIMessage(content='Command: UP', additional_kwargs={}, example=False)

In [5]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=chat_prompt)

In [6]:
situation = {'current':'t','up':'g','down':'g','left':'g','right':'w'}
chain.run(**situation)

'PICKUP_TREASURE'

# APIChain to interact with the game

We can pass a string with the api-description to APIChain. 
The README, has some curl examples, let's just pass that.

In [18]:
with open('README.md', 'r') as file:
    docs = file.read()
print(docs)

## langchain-treasurehunt
You find yourself in a dimly lit dungeon, with your torch barely providing enough light to see your immediate surroundings. The flickering flame casts eerie shadows on the damp stone walls. Your objective is clear: find the treasure hidden within this labyrinthine maze. But beware, the limited visibility adds to the challenge as you cautiously navigate through the darkness. Will you discover the treasure before your torch fades away completely? The race against time begins.

## Start game

`docker compose up -d`

## Observe game in browser

http://127.0.0.1:8089/

It will just show a list of buttons, that's fine.

## Let's try and solve the game step-by-step using LangChain and OpenAI

- Hint: Copy-paste `.env.sample` to `.env` and add your OpenAPI key 

1. `poetry install`
2. `poetry run jupyter notebook`
3. Open `langchain_treasurehunt.ipynb`
4. Run cells step by step manually

## Play the game using api
For each request, the api returns the surrounding cell

### Ask it to execute certain commands for you
Don't forget to observe the game status on http://localhost:8089/ while you execute these lines
- Hint: As the game grid is randomly chose, an 'invalid move' might occur here as the playe can't move there.

In [8]:
from langchain.chains import APIChain
llm_api = ChatOpenAI(model_name=model, temperature=0)
treasure_hunt_api = APIChain.from_llm_and_api_docs(llm_api, docs, verbose=True)

In [9]:
treasure_hunt_api.run('Start the game')



> Entering new APIChain chain...
http://localhost:8089/api/game?command=START
{"adjacent_cells":{"down":"W","left":"G","right":"W","up":"W"},"current_cell":"G","distance_to_treasure":2.23606797749979}


> Finished chain.


'The API response after starting the game includes the current cell, the distance to the treasure, and the adjacent cells in the up, down, left, and right directions.'

In [13]:
treasure_hunt_api.run('Go LEFT')



> Entering new APIChain chain...
http://localhost:8089/api/game?command=LEFT&response_format=current_cell
{"message":"Invalid move"}


> Finished chain.


'Going left is an invalid move.'

In [14]:
treasure_hunt_api.run('Go UP')



> Entering new APIChain chain...
http://localhost:8089/api/game?command=UP&fields=adjacent_cells.up,current_cell,distance_to_treasure
{"message":"Invalid move"}


> Finished chain.


'The response from the API for the question "Go UP" is "Invalid move".'

In [15]:
treasure_hunt_api.run('Pick up the treasure.')



> Entering new APIChain chain...
http://localhost:8089/api/game?command=PICK_UP
{"message":"No treasure at the current location"}


> Finished chain.


'The API response states that there is no treasure at the current location.'

## Agent: automate it by directly providing requests-tool to agent


In [27]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name=model, temperature=0)
tools = load_tools(["requests_all"])

prefix = f"""
You are on a 2D-grid and can see on the current and directly to adjacent cells. 
Goal: You need to find and pick up the treasure. Specify only the command for a given situation.

Rules of the game:
- Cell descriptions: W=wall, G=ground, T=treausre
- Available commands: UP DOWN LEFT RIGHT PICKUP_TREASURE
- You can only pick up the treasure if the current cell has the treasure on it and is not a ground.
- Do NOT attempt to pick up the treasure if it's in an adjacent cell. Wait until you have reached it.
- Do NOT walk towards adjacent cells with walls. Wall-cells are marked by the letter 'W'.
- Do NOT start the game again until you have found the treasure.

Keep in mind:
- Check your surroundings carefully to decide the next move.
- Remember on what cells you have been and include it in your thoughts.
- Solve the game step-by-step.
- Don't repeat the same invalid moves.

Call following API only using GET query-strings accordingly {docs}
"""
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_execution_time=90,
                         max_iterations=30)


In [28]:
agent.run(prefix+"\nStart the game and find the treasure")



> Entering new AgentExecutor chain...
We need to start the game and then find the treasure by making moves based on the adjacent cells and current cell information provided by the API.
Action: requests_get
Action Input: http://localhost:8089/api/game?command=START
Observation: {"adjacent_cells":{"down":"W","left":"G","right":"W","up":"W"},"current_cell":"G","distance_to_treasure":2.23606797749979}

Thought:We need to move up to get closer to the treasure.
Action: requests_get
Action Input: http://localhost:8089/api/game?command=UP
Observation: {"adjacent_cells":{"down":"W","left":"G","right":"W","up":"W"},"current_cell":"G","distance_to_treasure":2.23606797749979,"message":"Invalid move"}

Thought:We cannot move up as there is a wall. We need to move right instead.
Action: requests_get
Action Input: http://localhost:8089/api/game?command=RIGHT
Observation: {"adjacent_cells":{"down":"W","left":"G","right":"W","up":"W"},"current_cell":"G","distance_to_treasure":2.23606797749979,"messag

'Treasure found and picked up!'

# Is gpt-4 better?
`gpt-3.5-turbo` still walks in many walls and is not very good in interpreting distance. It also often tries to pick up the treasure exactly one field before reaching it. Maybe gpt-4 is better. Also using prompt engineering we could try and improve the agent.
Still in limited beta: https://platform.openai.com/docs/models/gpt-4